In [36]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

In [37]:
pd.options.display.max_columns = None

In [67]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [68]:
class NaiveBayes:
    def __init__(self, **kwargs):
        self.smoothing = kwargs.get('smoothing')
        self.model = kwargs.get('model') if kwargs.get('model') else 'bernoulli'

    def fit(self, X, y):
        self.X = X
        self.y = y

        if not y:
            self.y = self.X[:,-1]
            self.X = self.X[:,:-1]



        self.classes, self.class_counts = np.unique(self.y, return_counts=True)
        self.total_count = len(self.y)
        class_cols = []
        class_vals = [[]]

        for i, c in enumerate(self.classes):
            class_cols.append(f'p(y={c})')
            class_vals[0].append(np.round(self.class_counts[i]/self.total_count,4))

        self.prior_df = pd.DataFrame(class_vals, columns=class_cols)

        print('Class priors:')
        #prior_df.style.hide_index()
        print(self.prior_df.to_string(index=False))

        if self.is_bag_of_words():
            self.handle_bag_of_words()
        else:
            print('hi')

    def is_bag_of_words(self):
        return np.array(self.X).ndim == 1 \
               and np.array(self.X).dtype.kind in ['S', 'U'] and len(np.unique(self.X)) == self.total_count

    def handle_bag_of_words(self):
        words = [word for sentence in self.X for word in sentence.split()]

        word_counts = Counter(words)

        self.vocab = set(word_counts.keys())
        #print(self.vocab)
        if self.model == 'bernoulli':
            bag_data = [[int(word in sentence) for word in self.vocab] for sentence in self.X]
        else:
            bag_data = []
            for sentence in self.X:
                wc = Counter(sentence.split())
                bag_data.append([wc.get(word) if wc.get(word) else 0 for word in self.vocab ])
        #print(bag_data)
        df_bag = pd.DataFrame(bag_data, columns=self.vocab, index = self.X)
        df_bag['Class(Y)'] = self.y
        print()
        display(df_bag)
        
        self.probabilities = defaultdict(dict)
        
        for cls in self.classes:
            df_bag_cls = df_bag[df_bag['Class(Y)'] == cls]
            print(f'\n\nFor Y = {cls}')
            #print(df_bag_cls.sum().sum())
            display(df_bag_cls)
            idx_cls = df_bag_cls.index
            words_cls = [word for sentence in idx_cls for word in sentence.split()]
            for word in self.vocab:
                if self.model == 'bernoulli':
                    if self.smoothing:
                        print(f'\np({word}|y={cls}) = ({df_bag_cls[word].sum()} + 1)/({df_bag_cls.shape[0]} + 2)',
                              f'= {df_bag_cls[word].sum() + 1}/{df_bag_cls.shape[0] +2}',
                              f'= {np.round((df_bag_cls[word].sum() + 1)/(df_bag_cls.shape[0] +2), 4)}')
                        self.probabilities[cls][word] = np.round((df_bag_cls[word].sum() + 1)/(df_bag_cls.shape[0] +2), 4)
                    else:
                        print(f'\np({word}|y={cls}) = {df_bag_cls[word].sum()}/{df_bag_cls.shape[0]}',
                              f'= {np.round((df_bag_cls[word].sum())/(df_bag_cls.shape[0]),4)}')
                        self.probabilities[cls][word] = np.round((df_bag_cls[word].sum())/(df_bag_cls.shape[0]),4)
                else:                    
                    if self.smoothing:
                        print(f'\np({word}|y={cls}) = ({df_bag_cls[word].sum()} + 1)/({len(words_cls)} + {len(self.vocab)})',
                              f'= {df_bag_cls[word].sum() + 1}/{len(words_cls) +len(self.vocab)}',
                              f'= {np.round((df_bag_cls[word].sum() + 1)/(len(words_cls) +len(self.vocab)), 4)}')
                        self.probabilities[cls][word] = np.round((df_bag_cls[word].sum() + 1)/(len(words_cls) +len(self.vocab)), 4)
                    else:
                        print(f'\np({word}|y={cls}) = {df_bag_cls[word].sum()}/{len(words_cls)}',
                              f'= {np.round((df_bag_cls[word].sum())/(len(words_cls)),4)}')
                        self.probabilities[cls][word] = np.round((df_bag_cls[word].sum())/(len(words_cls)),4)
                        
            print(f'\n=====================================================================================\n')
            

In [69]:
NaiveBayes().fit(X = [1,2,3,4,5,6], y = [0,1,0,0,1,0])

Class priors:
 p(y=0)  p(y=1)
 0.6667  0.3333
hi


In [70]:
NaiveBayes(model = 'multi', smoothing = True).fit(X=['hi my ', 'hello name is', 'my name  name is sumil', 'hi hi hi where are you',
                    'what is this', 'nothing here'],
                 y=[0, 1, 0, 0, 1, 0])

Class priors:
 p(y=0)  p(y=1)
 0.6667  0.3333



,you,what,sumil,are,hi,is,nothing,name,my,hello,where,this,here,Class(Y)
hi my,0,0,0,0,1,0,0,0,1,0,0,0,0,0
hello name is,0,0,0,0,0,1,0,1,0,1,0,0,0,1
my name name is sumil,0,0,1,0,0,1,0,2,1,0,0,0,0,0
hi hi hi where are you,1,0,0,1,3,0,0,0,0,0,1,0,0,0
what is this,0,1,0,0,0,1,0,0,0,0,0,1,0,1
nothing here,0,0,0,0,0,0,1,0,0,0,0,0,1,0




For Y = 0
15


,you,what,sumil,are,hi,is,nothing,name,my,hello,where,this,here,Class(Y)
hi my,0,0,0,0,1,0,0,0,1,0,0,0,0,0
my name name is sumil,0,0,1,0,0,1,0,2,1,0,0,0,0,0
hi hi hi where are you,1,0,0,1,3,0,0,0,0,0,1,0,0,0
nothing here,0,0,0,0,0,0,1,0,0,0,0,0,1,0



p(you|y=0) = (1 + 1)/(15 + 13) = 2/28 = 0.0714

p(what|y=0) = (0 + 1)/(15 + 13) = 1/28 = 0.0357

p(sumil|y=0) = (1 + 1)/(15 + 13) = 2/28 = 0.0714

p(are|y=0) = (1 + 1)/(15 + 13) = 2/28 = 0.0714

p(hi|y=0) = (4 + 1)/(15 + 13) = 5/28 = 0.1786

p(is|y=0) = (1 + 1)/(15 + 13) = 2/28 = 0.0714

p(nothing|y=0) = (1 + 1)/(15 + 13) = 2/28 = 0.0714

p(name|y=0) = (2 + 1)/(15 + 13) = 3/28 = 0.1071

p(my|y=0) = (2 + 1)/(15 + 13) = 3/28 = 0.1071

p(hello|y=0) = (0 + 1)/(15 + 13) = 1/28 = 0.0357

p(where|y=0) = (1 + 1)/(15 + 13) = 2/28 = 0.0714

p(this|y=0) = (0 + 1)/(15 + 13) = 1/28 = 0.0357

p(here|y=0) = (1 + 1)/(15 + 13) = 2/28 = 0.0714




For Y = 1
8


,you,what,sumil,are,hi,is,nothing,name,my,hello,where,this,here,Class(Y)
hello name is,0,0,0,0,0,1,0,1,0,1,0,0,0,1
what is this,0,1,0,0,0,1,0,0,0,0,0,1,0,1



p(you|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526

p(what|y=1) = (1 + 1)/(6 + 13) = 2/19 = 0.1053

p(sumil|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526

p(are|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526

p(hi|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526

p(is|y=1) = (2 + 1)/(6 + 13) = 3/19 = 0.1579

p(nothing|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526

p(name|y=1) = (1 + 1)/(6 + 13) = 2/19 = 0.1053

p(my|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526

p(hello|y=1) = (1 + 1)/(6 + 13) = 2/19 = 0.1053

p(where|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526

p(this|y=1) = (1 + 1)/(6 + 13) = 2/19 = 0.1053

p(here|y=1) = (0 + 1)/(6 + 13) = 1/19 = 0.0526


